## Setup

In [1]:
!pip install -q transformers accelerate bitsandbytes \
  langchain sentence-transformers faiss-cpu langdetect huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
  

## Environment and Imports

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import gc
import json
import numpy as np
import faiss
from getpass import getpass

from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from huggingface_hub import login

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    pipeline,
)

## Hugging Face login

In [3]:
token = getpass('Enter your Hugging Face token (or press Enter to skip): ')
if token:
    login(token)

Enter your Hugging Face token (or press Enter to skip): ··········


## Paths

In [4]:
DATA_TXT = "/content/drive/MyDrive/GUVI_info_data.txt"
WORKDIR = "/content/drive/MyDrive/guvi_chatbot"
INDEX_PATH = f"{WORKDIR}/guvi_faiss.index"
CHUNKS_PATH = f"{WORKDIR}/chunks.txt"

os.makedirs(WORKDIR, exist_ok=True)

## Load Data

In [5]:
with open(DATA_TXT, "r", encoding="windows-1252") as f:
    raw_text = f.read()

print(f"Total characters: {len(raw_text)}")
print(raw_text[:500])

Total characters: 96178
GUVI — it’s an edtech platform based in India that started out of IIT Madras & IIM Ahmedabad.
They’re known for:
* Courses in vernacular languages (Tamil, Telugu, Hindi, etc.) so learners can study in their mother tongue.
* Focus on coding, data science, AI, and career prep.
* A Geek Network community where learners ask/answer questions.
* Challenges, hackathons, and Zen Class programs for intensive training.

Chapter 1: Introduction to GUVI

1.1 Overview of GUVI
GUVI (Grab, Understand, Verify, 


## Prepare Data

In [6]:
# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
)
chunks = text_splitter.split_text(raw_text)
print(f"✅ Total chunks created: {len(chunks)}")
print("\nExample Chunk:\n", chunks[0])

✅ Total chunks created: 468

Example Chunk:
 GUVI — it’s an edtech platform based in India that started out of IIT Madras & IIM Ahmedabad.
They’re known for:
* Courses in vernacular languages (Tamil, Telugu, Hindi, etc.) so learners can study in their mother tongue.
* Focus on coding, data science, AI, and career prep.


## Build FAISS

In [7]:
# Build embeddings + FAISS only if not already saved
if not (os.path.exists(INDEX_PATH) and os.path.exists(CHUNKS_PATH)):
    print("🔧 Building embeddings and FAISS index...")
    embedder_build = SentenceTransformer("all-MiniLM-L6-v2")  # CPU by default
    embeddings = embedder_build.encode(chunks, show_progress_bar=True)
    dim = embeddings[0].shape[0]

    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings, dtype=np.float32))

    faiss.write_index(index, INDEX_PATH)
    with open(CHUNKS_PATH, "w", encoding="utf-8") as f:
        for c in chunks:
            f.write(c.strip() + "\n")
    print("✅ FAISS index and chunks saved.")
else:
    print("ℹ️ Reusing existing FAISS index & chunks.")

🔧 Building embeddings and FAISS index...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

✅ FAISS index and chunks saved.


 ## Load FAISS + Chunks for Retrieval

In [8]:
index = faiss.read_index(INDEX_PATH)
with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    chunks = [line.strip() for line in f.readlines()]

# Light embedding model for queries (CPU)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

## Load Models with Split Memory

In [9]:
!pip install -q transformers accelerate langdetect sentence-transformers faiss-cpu

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [10]:
mistral_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
https://colab.research.google.com/drive/1PyPVrZ2HsbJRmmpJtERtjqIHasSZmF7M?authuser=2#scrollTo=Load_FAISS_Chunks_for_Retrieval
mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_model_name)

mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral_model_name,
    device_map="auto",   # ✅ split across GPU/CPU
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    offload_folder="offload_mistral"  # ✅ safely offload excess weights
)

# ❌ Do NOT pass device=...
generator = pipeline(
    "text-generation",
    model=mistral_model,
    tokenizer=mistral_tokenizer
)

print("✅ Mistral loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Mistral loaded successfully!


## Load FAISS + Embedding Model

In [11]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load FAISS index
index = faiss.read_index("/content/drive/MyDrive/guvi_chatbot/guvi_faiss.index")

# Load chunks
with open("/content/drive/MyDrive/guvi_chatbot/chunks.txt", "r", encoding="utf-8") as f:
    chunks = [line.strip() for line in f.readlines()]

# Embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

print("✅ FAISS + Embedding model loaded!")


✅ FAISS + Embedding model loaded!


## Load NLLB (Translation)


In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

nllb_model_name = "facebook/nllb-200-distilled-600M"

nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)

# Force entire model to CPU
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(
    nllb_model_name,
    device_map={"": "cpu"},   # entire model on CPU
    torch_dtype=torch.float32 # CPU-friendly dtype
)

# ❌ DO NOT pass device argument here
translator = pipeline(
    "translation",
    model=nllb_model,
    tokenizer=nllb_tokenizer
)

print("✅ NLLB loaded successfully on CPU!")



Device set to use cpu


✅ NLLB loaded successfully on CPU!


## Language Code Map

In [15]:
lang_code_map = {
    # Indian
    "ta": "tam_Taml", "hi": "hin_Deva", "te": "tel_Telu",
    "ml": "mal_Mlym", "kn": "kan_Knda", "bn": "ben_Beng", "mr": "mar_Deva",
    # Foreign
    "fr": "fra_Latn", "de": "deu_Latn", "ko": "kor_Hang",
    "zh": "zho_Hans", "zh-cn": "zho_Hans", "ja": "jpn_Jpan",
    # English
    "en": "eng_Latn"
}

print("✅ Language map ready!")


✅ Language map ready!


## RAG Prompt + Chatbot

In [16]:
def build_rag_prompt(context_chunks, question):
    context_text = "\n".join(context_chunks)
    return f"""You are a multilingual chatbot for GUVI. The user may ask questions in any language. Answer in English, and only based on the context given.

### Context:
{context_text}

### Question:
{question}

### Answer:"""

from langdetect import detect

def guvi_chatbot(user_input, top_k=3, back_translate=True, max_new_tokens=200):
    # 1) Detect language
    try:
        lang = detect(user_input)
    except Exception:
        lang = "en"
    src_lang = lang_code_map.get(lang, "eng_Latn")  # fallback

    # 2) Translate to English if needed
    if lang != "en":
        print(f"🌐 Detected language: {lang} → Translating to English")
        translated = translator(
            user_input,
            src_lang=src_lang,
            tgt_lang="eng_Latn",
            max_length=1024
        )[0]['translation_text']
    else:
        translated = user_input

    # 3) Retrieve top_k chunks via FAISS
    q_embed = embedder.encode([translated])
    D, I = index.search(np.array(q_embed, dtype=np.float32), top_k)
    context_chunks = [chunks[i] for i in I[0]]

    # 4) Build prompt and generate with Mistral
    prompt = build_rag_prompt(context_chunks, translated)
    out = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        pad_token_id=mistral_tokenizer.eos_token_id
    )
    generated = out[0]["generated_text"].split("### Answer:")[-1].strip()

    # 5) Translate back to user language if needed
    if lang != "en" and back_translate:
        try:
            final_answer = translator(
                generated,
                src_lang="eng_Latn",
                tgt_lang=src_lang,
                max_length=1024
            )[0]['translation_text']
            print("\n🤖 Answer (translated back to your language):\n")
            return final_answer
        except Exception as e:
            print(f"⚠️ Back-translation failed: {e}")
            return generated

    print("\n🤖 Answer:\n")
    return generated

## Test

In [18]:
print(guvi_chatbot("What is special about GUVI compared to other learning platforms?")) #English


🤖 Answer:

GUVI sets itself apart by partnering with reputed universities like Jain University, enabling learners to earn academic credits alongside practical skills. Additionally, GUVI integrates with LinkedIn API to update learner profiles directly, boosting professional visibility.


In [22]:
# “Will I get a job after learning at GUVI?”

print(guvi_chatbot("GUVIயில் கற்றுக்கொண்ட பிறகு வேலை வாய்ப்பு கிடைக்குமா?")) #Tamil

🌐 Detected language: ta → Translating to English

🤖 Answer (translated back to your language):

GUVI இல் படிக்கும் போது, நீங்கள் ஒரு புகழ்பெற்ற கல்வி நிறுவனமான ஜைன் பல்கலைக்கழகத்திலிருந்து அறிவைப் பெறுவீர்கள், மேலும் GUVI இன் மேம்பட்ட திட்டங்கள் மூலம் தொழில் தயாரான திறன்களைப் பெறுவீர்கள். இந்த கலப்பின மாதிரி உங்கள் வேலைவாய்ப்பு திறனை அதிகரிக்க வடிவமைக்கப்பட்டுள்ளது. இருப்பினும், வேலைவாய்ப்பு பெறுவது இறுதியில் உங்கள் தனிப்பட்ட செயல்திறன், வேலை சந்தை நிலைமைகள் மற்றும் குறிப்பிட்ட பதவிகளுக்கு விண்ணப்ப செயல்முறையைப் பொறுத்தது. வேலை நேர்காணல்கள் மற்றும் விண்ணப்பங்களுக்கு மாணவர்களுக்குத் தயாராக உதவ GUVI ஆதரவையும் வளங்களையும் வழங்குகிறது.


In [20]:
# In how much time are the courses on GUVI completed?

print(guvi_chatbot("GUVI पर मिलने वाले कोर्स कितने समय में पूरे हो जाते हैं?")) #Hindi

🌐 Detected language: hi → Translating to English

🤖 Answer (translated back to your language):

हां, GUVI उपयोगकर्ता प्रमाणीकरण के लिए OAuth 2.0 और एकल साइन-ऑन (SSO) का समर्थन करता है। जबकि साइन-ऑन प्रक्रिया के लिए विशिष्ट भाषाओं का संदर्भ में उल्लेख नहीं किया गया है, अधिकांश प्रमाणीकरण प्रणाली कई भाषाओं के लिए विकल्प प्रदान करती हैं। साइन-ऑन प्रक्रिया के लिए भाषा विकल्पों के बारे में अधिक विस्तृत जानकारी के लिए, मैं सीधे GUVI की सहायता टीम से संपर्क करने की सलाह देता हूं। वे आपको भाषा समर्थन के बारे में सबसे सटीक और अद्यतित जानकारी प्रदान करने में सक्षम होंगे


In [21]:
# Does GUVI provide a certificate after completing courses?

print(guvi_chatbot("GUVI లో కోర్సులు పూర్తి చేసిన తర్వాత సర్టిఫికేట్ ఇస్తారా?")) #Telugu

🌐 Detected language: te → Translating to English

🤖 Answer (translated back to your language):

అవును, GUVI లో కోర్సులు పూర్తయిన తర్వాత మీకు సర్టిఫికేట్ లభిస్తుంది. ఈ సర్టిఫికేట్ మీరు ఎంచుకున్న రంగంలో అవసరమైన జ్ఞానం మరియు నైపుణ్యాలను పొందారని సూచిస్తుంది, ఇది మిమ్మల్ని పరిశ్రమకు సిద్ధంగా చేస్తుంది. జైన్ విశ్వవిద్యాలయం వంటి విశ్వవిద్యాలయాలతో భాగస్వామ్యం కారణంగా GUVI యొక్క కోర్సులు అకాడెమిక్ బరువు కలిగి ఉన్నప్పటికీ, ప్రాధమిక దృష్టి ప్రాక్టికల్, హ్యాండ్స్-ఆన్ అనుభవాన్ని నిర్ధారించడం.


In [23]:
# Can the classes on GUVI be recorded and watched again?

print(guvi_chatbot("GUVIയിലെ ക്ലാസുകൾ റെക്കോർഡ് ചെയ്ത് വീണ്ടും കാണാൻ കഴിയുമോ?")) #Malayalam

🌐 Detected language: ml → Translating to English

🤖 Answer (translated back to your language):

ഏതൊരു ആശയക്കുഴപ്പത്തിനും ക്ഷമിക്കണം, പക്ഷേ GUVI ൽ റെക്കോർഡിംഗ് ക്ലാസുകളെക്കുറിച്ച് നൽകിയിരിക്കുന്ന സന്ദർഭം ഒന്നും പരാമർശിക്കുന്നില്ല. എന്നിരുന്നാലും, ഒരു പഠിതാവെന്ന നിലയിൽ, നിങ്ങളുടെ ലിങ്ക്ഡ്ഇൻ പ്രൊഫൈൽ GUVI വഴി നിങ്ങളുടെ സർട്ടിഫിക്കേഷൻ നിലയുമായി നേരിട്ട് അപ്ഡേറ്റ് ചെയ്യാമെന്ന് എനിക്ക് സ്ഥിരീകരിക്കാൻ കഴിയും, നിങ്ങളുടെ പ്രൊഫഷണൽ വികസനം സാധ്യതയുള്ള തൊഴിലുടമകൾക്ക് ദൃശ്യമാകുമെന്ന് ഉറപ്പാക്കുന്നു. ഉറപ്പാക്കുക, GUVI ഡാറ്റ കൈകാര്യം ചെയ്യുന്നതിനുള്ള ഇന്ത്യൻ ഐടി നിയമവും ജിഡിപിആർ പോലുള്ള തത്വങ്ങളും പാലിക്കുന്നു, നിങ്ങളുടെ സ്വകാര്യത പരിരക്ഷിക്കപ്പെടുന്നുവെന്ന് ഉറപ്പാക്കുന്നു.


In [24]:
# Do we get job opportunities after learning at GUVI?

print(guvi_chatbot("GUVIಯಲ್ಲಿ ಕಲಿತ ನಂತರ ಕೆಲಸದ ಅವಕಾಶ ಸಿಗುತ್ತದೆಯೇ?")) #Kannada

🌐 Detected language: kn → Translating to English

🤖 Answer (translated back to your language):

GUVI ಯಲ್ಲಿ ಅಧ್ಯಯನ ಮಾಡಿದ ನಂತರ ನಿಮಗೆ ಉದ್ಯೋಗ ದೊರೆಯುತ್ತದೆ ಎಂದು ನಾನು ಭರವಸೆ ನೀಡಲಾರೆ, ಆದರೆ ಜೈನ್ ವಿಶ್ವವಿದ್ಯಾಲಯ ಮತ್ತು ಸ್ವಯಂಚಾಲಿತ ಮೌಲ್ಯಮಾಪನ ವ್ಯವಸ್ಥೆಯೊಂದಿಗಿನ ವಿಶ್ವವಿದ್ಯಾಲಯದ ಸಹಭಾಗಿತ್ವವು ಶೈಕ್ಷಣಿಕ ತೂಕ ಮತ್ತು ಉದ್ಯಮದ ಸನ್ನದ್ಧತೆಯನ್ನು ಒದಗಿಸುವ ಮೂಲಕ ಉದ್ಯೋಗ ಪಡೆಯುವ ನಿಮ್ಮ ಅವಕಾಶಗಳನ್ನು ಹೆಚ್ಚಿಸುತ್ತದೆ.


In [25]:
# How much job opportunity is available after learning from GUVI?

print(guvi_chatbot("GUVI থেকে শেখার পর চাকরির সুযোগ কতটা পাওয়া যায়?")) #Bengali

🌐 Detected language: bn → Translating to English

🤖 Answer (translated back to your language):

আমি GUVI-তে পড়াশোনার পর কাজের সুযোগের সঠিক সংখ্যা বলতে পারি না। তবে বিশ্ববিদ্যালয়ের অংশীদারিত্ব এবং শিল্প-ভিত্তিক পাঠ্যক্রমের লক্ষ্য শিক্ষার্থীদের কর্মসংস্থান বৃদ্ধি করা।


In [26]:
# What are the advantages of GUVI for beginner students?

print(guvi_chatbot("Quels sont les avantages de GUVI pour les étudiants débutants ?")) #French

🌐 Detected language: fr → Translating to English

🤖 Answer (translated back to your language):

GUVI offre plusieurs avantages pour les débutants, quel que soit le langage de votre question. Voici quelques avantages clés: 1. L'apprentissage interactif: GUVI offre une expérience d'apprentissage interactive via sa plate-forme, permettant aux débutants de comprendre les concepts plus efficacement. 2. Projets du monde réel: GUVI offre des projets du monde réel pour aider les débutants à acquérir une expérience pratique et à bâtir une base solide dans diverses technologies. 3. Prix abordables: GUVI offre des options de prix abordables, ce qui rend accessible aux débutants qui commencent leur parcours d'apprentissage. 4. Certifications reconnues par l'industrie: GUVI offre des certifications avec des organisations leaders de l'industrie comme Microsoft, Google et AWS qui peuvent stimuler vos perspectives de carrière. 5. Mises à jour directes: GUVI propose des intégrations avec des plateforme

In [27]:
# What special courses does GUVI offer?

print(guvi_chatbot("Welche besonderen Kurse bietet GUVI an?")) #German

🌐 Detected language: de → Translating to English


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



🤖 Answer (translated back to your language):

GUVI bietet eine Reihe von Kursen an, von denen einige spezialisiert sind. Ohne eine bestimmte Sprache oder einen zusätzlichen Kontext zu kennen, kann ich jedoch keine endgültige Antwort darauf geben, welche Kurse als "speziell" gelten. Die von GUVI abgedeckten Kurse können Themen im Zusammenhang mit der Android- und iOS-App-Entwicklung umfassen, mit dem Schwerpunkt auf Offline-Video-Wiedergabe. Darüber hinaus können GUVI-Kurse als Partner von AICTE und NEAT akademisches Gewicht tragen und Studenten für die Bereitschaft der Branche vorbereiten. Für die genauesten und aktuellsten Informationen empfehle ich, die GUVI-Website zu besuchen oder direkt mit ihrem Support-Team in Verbindung zu treten.


In [28]:
# Does learning at GUVI help with getting a job?

print(guvi_chatbot("GUVI에서 배우면 취업에 도움이 되나요?")) #Korean

🌐 Detected language: ko → Translating to English

🤖 Answer (translated back to your language):

제인 대학교와의 파트너십을 통해 GUVI에서 공부하는 것은 학술적 인정을 제공 할 수 있지만, 플랫폼은 또한 산업 준비에 초점을 맞추고 있습니다. 이것은 제공되는 과정이 기술 산업에서 일자리를 준비하는 데 도움이 될 수 있음을 의미합니다. 그러나 궁극적으로 일자리를 얻는 것은 개인 기술, 면접시에서의 성능 및 현재의 고용 시장 조건과 같은 다양한 요소에 달려 있습니다.


In [29]:
# Can one find a job after studying at GUVI?

print(guvi_chatbot("在GUVI学习后能找到工作吗？")) #Chinese

🌐 Detected language: zh-cn → Translating to English

🤖 Answer (translated back to your language):

是的,GUVI的课程,其学术重量来自贾因大学和行业重点,旨在为学生做好就业市场的准备.然而,获得工作最终取决于个人表现和市场条件.GUVI提供职业服务和行业互动,以帮助学生提高他们的就业能力.


In [30]:
# Does what you learn at GUVI help in finding a job?

print(guvi_chatbot("GUVIで学んだことは就職に役立ちますか？")) #Japanese

🌐 Detected language: ja → Translating to English

🤖 Answer (translated back to your language):

はい,GUVIのコースは,ジャイン大学と業界と協力して設計されており,学術的な深さと業界準備の両方を保証しています.この組み合わせにより,私たちの卒業生は雇用市場で非常に競争力があります.
